In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mpl
from matplotlib import pyplot as plt

from scipy.stats import zscore

from collections import defaultdict, Counter

import re

import scipy.spatial as sp
import scipy.cluster.hierarchy as hc

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [2]:
rppa = pd.read_hdf("../data/CCLE_RPPA_20181003.hdf",key="rppa")

avana = pd.read_hdf("../data/Achilles_gene_effect.hdf",key="avana")
drive = pd.read_hdf("../data/D2_DRIVE_gene_dep_scores.hdf",key="drive")
achilles = pd.read_hdf("../data/D2_Achilles_gene_dep_scores.hdf",key="achilles")

rppa_info = pd.read_excel("../data/41586_2019_1186_MOESM17_ESM.xlsx",sheet_name="RPPA_Ab_information")


In [7]:
avana_name_map = dict(zip(["_".join(x.split("_")[:-1]) for x in avana.columns],avana.columns))
drive_name_map = dict(zip(["_".join(x.split("_")[:-1]) for x in drive.columns],drive.columns))
achilles_name_map = dict(zip(["_".join(x.split("_")[:-1]) for x in achilles.columns],achilles.columns))

In [8]:
rppa_info["avana_gene"] = rppa_info["Target_Genes"].apply(lambda x: avana_name_map.get(x,np.nan))
rppa_info["drive_gene"] = rppa_info["Target_Genes"].apply(lambda x: drive_name_map.get(x,np.nan))
rppa_info["a_gene"] = rppa_info["Target_Genes"].apply(lambda x: a_name_map.get(x,np.nan))

,Antibody_Name,Target_Genes,Validation_Status,Company,Catalog_Number
0,14-3-3_beta,YWHAB,Validated,Santa Cruz,sc-628
1,14-3-3_epsilon_Caution,YWHAE,Caution,Santa Cruz,sc-23957
2,14-3-3_zeta,YWHAZ,Validated,Santa Cruz,sc-1019
3,4E-BP1,EIF4EBP1,Validated,CST,9452
4,4E-BP1_pS65,EIF4EBP1,Validated,CST,9456
5,4E-BP1_pT37_T46,EIF4EBP1,Validated,CST,9459
6,4E-BP1_pT70,EIF4EBP1,Validated,CST,9455
7,53BP1,TP53BP1,Validated,CST,4937
8,A-Raf_pS299_Caution,ARAF,Caution,CST,4431
9,ACC_pS79,ACACA ACACB,Validated,CST,3661
